# Week 10 Discussion: LSTM in Practice

## Objectives

In this week's discussion, we will learn:
* LSTM in Part-of-Speech Tagging

## Source

This discussion has been adapted from one of the tutorials in _Pytorch_ which has been written by _Robert Guthrie_. 
The source can be found [here](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html).

We now start by first importing the _Pytorch_ library. Since some parts are used frequently, we import them individually. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

To get the same results in multiple runs, we set the seed:

In [2]:
torch.manual_seed(1)

## Defining the First LSTM

We define an LSTM in the line below. Input and output dimensions are both 3.

In [3]:
lstm = nn.LSTM(3, 3)

we create a random sequence of inputs which consists of 5 3D data points.

In [4]:
inputs = [torch.randn(1, 3) for _ in range(5)]

Printing the inputs we have:

In [5]:
print (inputs)

[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]


We now initialize the hidden state:

In [6]:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

We now pass the LSTM into the sequence in a for loop:

In [7]:
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
print (out)
print (hidden)

tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<CatBackward>)
(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<ViewBackward>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<ViewBackward>))


In order to see what the function _view_ does, we print out the first element of the input before and after applying it:

In [8]:
print (inputs[0].shape, inputs[0].view(1, 1, -1).shape)

torch.Size([1, 3]) torch.Size([1, 1, 3])


We can also pass the entire sequence all at once.

We first get the input in the right format:

In [9]:
inputs = torch.cat(inputs)
print (inputs)
print (inputs.shape)

tensor([[-0.5525,  0.6355, -0.3968],
        [-0.6571, -1.6428,  0.9803],
        [-0.0421, -0.8206,  0.3133],
        [-1.1352,  0.3773, -0.2824],
        [-2.5667, -1.4303,  0.5009]])
torch.Size([5, 3])


In [10]:
inputs = inputs.view(len(inputs), 1, -1)
print (inputs)
print (inputs.shape)

tensor([[[-0.5525,  0.6355, -0.3968]],

        [[-0.6571, -1.6428,  0.9803]],

        [[-0.0421, -0.8206,  0.3133]],

        [[-1.1352,  0.3773, -0.2824]],

        [[-2.5667, -1.4303,  0.5009]]])
torch.Size([5, 1, 3])


We reinitialize the hidden state:

In [11]:
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3)) 

By feeding the data to the model all at once, we have:

In [12]:
out, hidden = lstm(inputs, hidden)

out     (first returned value): all hidden states in the sequence
<br>
hidden (second returned value): most recent hidden state

In [13]:
print (out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


## An LSTM for Part-of-Speech Tagging

Now we want to apply LSTM to a small dataset which is the part of speech in two sentences. We start by defining a training set:

In [14]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In order to see what _split()_ does, we print out the output:

In [15]:
print ("The dog ate the apple".split())

['The', 'dog', 'ate', 'the', 'apple']


we first make the labels numerical as always. For this we define a function which inputs a sequence and outputs a tensor which is the indices of that sequence based on a dictionary of indices:

In [16]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

Now we define the dictionaries used for transforming the words and tags into indices:

In [17]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

Now we set the embedding dimension and the hidden dimension of the LSTM:

In [18]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Here we define a class. This class should implement the constructor of the class in addition to the class method _forward_. 

The input to the LSTM is the word embeddings and outputs hidden states with dimensionality _hidden dim_.

Additionally, we define a linear layer to map the hidden space to the output space (tag)

In [19]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

We now define the model, the loss function, and the optimizer:

In [20]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

Before doing the training, we test the model to see the tag scores. Since we are not training, add torch.no_grad() wrapper to set all the requires_grad flag to false temporarily.

In [21]:
print (training_data[0][0])
print (training_data[0][1])
print (prepare_sequence(training_data[0][1], tag_to_ix))
print ('\n')
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print (tag_scores)
    print (tag_scores.argmax(dim=1))

['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
tensor([0, 1, 2, 0, 1])


tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([2, 2, 2, 2, 0])


Now we do the training. Since Pytorch accumulates the gradients, we need to zero them out before each instance. 

In [22]:
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()

        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

Passing the first sentence through the model again, we have:

In [23]:
print (training_data[0][0])
print (training_data[0][1])
print (prepare_sequence(training_data[0][1], tag_to_ix))
print ('\n')
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print (tag_scores)
    print (tag_scores.argmax(dim=1))

['The', 'dog', 'ate', 'the', 'apple']
['DET', 'NN', 'V', 'DET', 'NN']
tensor([0, 1, 2, 0, 1])


tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])
tensor([0, 1, 2, 0, 1])
